In [28]:
import numpy as np
import tensorflow as tf

from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Activation, Lambda
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.applications.resnet50 import identity_block, conv_block
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras import optimizers

In [ ]:
def load_processed_fish(pathname, target_size):
    num_images = 3000
    label_bounds = [0, 199, 1910, 2641, 2758, 2933, 3000] #not using other fish
    label_counts = [label_bounds[i+1]-label_bounds[i] for i in range(len(label_bounds)-1)]
    
    fish_images = []
    for label in range(len(label_counts)):
        images = np.empty((label_counts[label], 224, 224, 3))
        for i in range(label_counts[label]):
            images[i,:,:,:] = skimage.io.imread(os.path.join(pathname, 
                "img_{0}label_{1}.jpg".format(i + label_bounds[label], label)))
        resize_image_set(images, target_size)
        fish_images.append(images)

    X = np.empty((6*target_size, 224, 224, 3))
    index = 0
    for fish in fish_images:
        for i in range(len(fish)):
            X[index] = fish[i]
            index += 1


    #.reshape((-1,224,224,3))
    y = []
    for i in range(len(label_counts)):
        y += [i] * target_size
    y = np.array(y)

    print "Loaded training images with shape {0}.".format(X.shape)
    print "Loaded training labels with shape {0}.".format(y.shape)
    np.save('samples.npy', X)
    np.save('labels.npy', y)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    shuffled_images = X[indices]
    shuffled_labels = y[indices]
    split = int(X.shape[0]*.9)
    training_images = shuffled_images[:split]
    training_labels = shuffled_labels[:split]

    val_images = shuffled_images[split:]
    val_labels = shuffled_labels[split:]

    return training_images, training_labels, val_images, val_labels

In [31]:
model = Sequential()
model.add(Dense(output_dim=64, input_dim=100))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

trn_path = 'train/'
train_datagen = ImageDataGenerator()
trn_gen = train_datagen.flow_from_directory(trn_path, target_size=(224,224), batch_size=50,
                                                    class_mode='categorical', shuffle=True)

# model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)
# loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)

Found 3777 images belonging to 8 classes.


In [51]:
px_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def preprocess(x):
    x = x - px_mean
    return tf.reverse(x, [False, True, False, False])

In [54]:
size = (224, 224)
n_classes = 2
lr = 0.001
batch_size = 64
dropout = 0.5
weights_file = 'vgg16.h5'

model = Sequential()
model.add(Lambda(preprocess, input_shape=(3,224,224)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='softmax'))

model.load_weights(weights_file)
model.pop(); model.pop(); model.pop()

for layer in model.layers:
    layer.trainable = False

model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adadelta", metrics=["accuracy"])

ValueError: Filter must not be larger than the input: Filter: (2, 2) Input: (1, 112)

In [23]:
px_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def preprocess(x):
    x = x - px_mean
    return tf.reverse(x, [False, True])

In [19]:
# paths
train_path = '/train/'
# test_path = '/test/'
saved_model_path = '/models/'

# data
batch_size = 16
nb_split_train_samples = 3277
nb_full_train_samples = 3785
nb_valid_samples = 500
# nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_runs = 5
nb_epoch = 30
nb_aug = 5
dropout = 0.4
clip = 0.01
use_val = False
archs = ["vggbn"]

models = {
    "vggbn": Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=0.001,
                           batch_size=batch_size, dropout=dropout),
#     "inception": Inception(size=(299, 299), n_classes=nb_classes,
#                            lr=0.001, batch_size=batch_size),
#     "resnet": Resnet50(size=(270, 480), n_classes=nb_classes, lr=0.001,
#                     batch_size=batch_size, dropout=dropout)
}

In [20]:
def train(parent_model, model_str):
    parent_model.build()    
    model_fn = saved_model_path + '{val_loss:.2f}-loss_{epoch}epoch_' + model_str
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)
    
    if use_val:
        parent_model.fit_val(split_train_path, valid_path, nb_trn_samples=nb_split_train_samples, 
                             nb_val_samples=nb_valid_samples, nb_epoch=nb_epoch, callbacks=[ckpt], aug=nb_aug)

        model_path = max(iglob(saved_model_path + '*.h5'), key=os.path.getctime)
        return model_path
    
    model_fn = saved_model_path + '{}epoch_'.format(nb_epoch) + model_str
    parent_model.fit_full(full_train_path, nb_trn_samples=nb_full_train_samples, nb_epoch=nb_epoch, aug=nb_aug)
    model.save_weights(model_fn)
    del parent_model.model 
    
    return model_fn

In [21]:
def train_all():    
    model_paths = {
        "vggbn": [],
        "inception": [],
        'resnet': [],
    }
    
    for run in range(nb_runs):
        print("Starting Training Run {0} of {1}...\n".format(run+1, nb_runs))
        aug_str = "aug" if nb_aug else "no-aug"
        
        for arch in archs:
            print("Training {} model...\n".format(arch))
            model = models[arch]
            model_str = "{0}x{1}_{2}_{3}lr_run{4}_{5}.h5".format(model.size[0], model.size[1], aug_str,
                                                                 model.lr, run, arch)
            model_path = train(model, model_str)
            model_paths[arch].append(model_path)
        
    print("Done.") 
    return model_paths

In [24]:
model_paths = train_all()

Starting Training Run 1 of 5...

Training vggbn model...



ValueError: Shape (?, 3, 270, 480) must have rank 2